In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 5
%autosave 15

import pandas as pd
import scipy as sp
import numpy as np
import matplotlib.pyplot as plt
import math

import sklearn
from sklearn import datasets
from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import cross_val_score
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.svm import SVC, SVR
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.grid_search import GridSearchCV

Autosaving every 15 seconds


In [2]:
train_data = pd.read_csv('Data/train.csv')
train_data.describe()

id       feat_1        feat_2        feat_3        feat_4  \
count  61878.000000  61878.00000  61878.000000  61878.000000  61878.000000   
mean   30939.500000      0.38668      0.263066      0.901467      0.779081   
std    17862.784315      1.52533      1.252073      2.934818      2.788005   
min        1.000000      0.00000      0.000000      0.000000      0.000000   
25%    15470.250000      0.00000      0.000000      0.000000      0.000000   
50%    30939.500000      0.00000      0.000000      0.000000      0.000000   
75%    46408.750000      0.00000      0.000000      0.000000      0.000000   
max    61878.000000     61.00000     51.000000     64.000000     70.000000   

             feat_5        feat_6        feat_7        feat_8        feat_9  \
count  61878.000000  61878.000000  61878.000000  61878.000000  61878.000000   
mean       0.071043      0.025696      0.193704      0.662433      1.011296   
std        0.438902      0.215333      1.030102      2.255770      3.474822   
min        0.000000      0.000000      0.000000      0.000000      0.000000   
25%        0.000000      0.000000      0.000000      0.000000      0.000000   
50%        0.000000      0.000000      0.000000      0.000000      0.000000   
75%        0.000000      0.000000      0.000000      1.000000      0.000000   
max       19.000000     10.000000     38.000000     76.000000     43.000000   

           ...            feat_84       feat_85       feat_86       feat_87  \
count      ...       61878.000000  61878.000000  61878.000000  61878.000000   
mean       ...           0.070752      0.532306      1.128576      0.393549   
std        ...           1.151460      1.900438      2.681554      1.575455   
min        ...           0.000000      0.000000      0.000000      0.000000   
25%        ...           0.000000      0.000000      0.000000      0.000000   
50%        ...           0.000000      0.000000      0.000000      0.000000   
75%        ...           0.000000      0.000000      1.000000      0.000000   
max        ...          76.000000     55.000000     65.000000     67.000000   

            feat_88       feat_89       feat_90       feat_91       feat_92  \
count  61878.000000  61878.000000  61878.000000  61878.000000  61878.000000   
mean       0.874915      0.457772      0.812421      0.264941      0.380119   
std        2.115466      1.527385      4.597804      2.045646      0.982385   
min        0.000000      0.000000      0.000000      0.000000      0.000000   
25%        0.000000      0.000000      0.000000      0.000000      0.000000   
50%        0.000000      0.000000      0.000000      0.000000      0.000000   
75%        1.000000      0.000000      0.000000      0.000000      0.000000   
max       30.000000     61.000000    130.000000     52.000000     19.000000   

            feat_93  
count  61878.000000  
mean       0.126135  
std        1.201720  
min        0.000000  
25%        0.000000  
50%        0.000000  
75%        0.000000  
max       87.000000  

[8 rows x 94 columns]

In [4]:
clf = RandomForestClassifier(n_estimators=400, max_depth=15, min_samples_split=12, min_samples_leaf=3, max_features=20)
rf_best_params = {"n_estimators": [450,500], 
                  "max_depth": [10,15,20], 
                  "min_samples_split": [6,13], 
                  "min_samples_leaf": [1,3],
                  "max_features": [20,30,40]}

In [3]:
train_data = train_data.drop("id", axis = 1)
def convert_target(x):
    return int(x["target"].split('_')[1])
train_data["target"] = train_data.apply(convert_target, axis = 1)

In [4]:
from sklearn.metrics import log_loss, accuracy_score
import datetime, time
def test_method(method, a_train, a_test, b_train, b_test):
    print method.__class__
    method.fit(a_train, b_train)
    print log_loss(b_train, method.predict_proba(a_train))

def test_methods(methods, a_train, a_test, b_train, b_test):
    for method in methods:
        test_method(method, a_train, a_test, b_train, b_test)

def test_models(data):
    target = data.target
    train = data.drop('target', axis = 1)
    X_train, X_test, y_train, y_test = train_test_split(train, target, test_size = 0.2, random_state = 42)
    methods = [RandomForestClassifier(n_estimators = 100)]
    test_methods(methods, X_train, X_test, y_train, y_test)

def get_best_params(method, params, data):
    target = data.target
    train = data.drop('target', axis = 1)
    X_train, X_test, y_train, y_test = train_test_split(train, target, test_size = 0.2, random_state = 42)
    errors_train, errors_test = aggregate_train_test_errors(method, params, X_train, y_train, X_test, y_test, [log_loss], print_flag=True)
    plot_train_test_errors(method, params, errors_train, errors_test)
    
def print_score(model, metric, X_train, y_train, X_test, y_test):
        print("Model: %s Metric: %s On test: %f" % (model.__class__.__name__, metric.__name__, metric(y_test, model.predict_proba(X_test))))
        
def print_scores(model, metrics, X_train, y_train, X_test, y_test, parameter = None):
    for metric in metrics:
        if parameter != None:
            print("%s %s" % (parameter[0], parameter[1]))
            print_score(model, metric, X_train, y_train, X_test, y_test)
        else:
            print_score(model, metric, X_train, y_train, X_test, y_test)
    
def aggregate_train_test_errors(model, params, X_train, y_train, X_test, y_test, metrics = [], print_flag = False):
    errors_train = {key: {'params': params[key], 'values': []} for key in params.keys()}
    errors_test = {key: {'params': params[key], 'values': []}  for key in params.keys()}
    base_parameters = model.get_params()
    start_time = time.time()
    prev_time = start_time
    for key, values in params.iteritems():
        if isinstance(values, list):
            for value in values:
                model.set_params(**{key: value})
                model.fit(X_train, y_train)
                cur_time = time.time()
                from_start = datetime.datetime.fromtimestamp(cur_time - start_time + time.timezone).strftime('%H:%M:%S')
                from_last = datetime.datetime.fromtimestamp(cur_time - prev_time + time.timezone).strftime('%H:%M:%S')
                print from_start + " (" + from_last + ")"
                prev_time = cur_time
                errors_train[key]['values'].append(model.score(X_train, y_train))
                errors_test[key]['values'].append(model.score(X_test, y_test))
                if print_flag == True and metrics:
                    print_scores(model, metrics, X_train, y_train, X_test, y_test, (key, value))
            model.set_params(**base_parameters)
        else:
            raise Exception("Take only list of parameters!")   
    return errors_train, errors_test

def plot_train_test_errors(model, model_params, errors_train, errors_test, save = None):
    fig, axes = plt.subplots(ncols=len(model_params))
    fig.set_size_inches((15,9))
    for ind, param in enumerate(model_params.keys()):
        if isinstance(errors_train[param]['params'][0], str):
            axes[ind].plot(range(len(errors_train[param]['params'])), errors_train[param]['values'], label = 'on train')
            axes[ind].plot(range(len(errors_train[param]['params'])), errors_test[param]['values'], label = 'on test')
            axes[ind].xaxis.set_ticks(range(len(errors_train[param]['params'])), errors_train[param]['params'])
        else:            
            axes[ind].plot(errors_train[param]['params'], errors_train[param]['values'], label = 'on train')
            axes[ind].plot(errors_train[param]['params'], errors_test[param]['values'], label = 'on test')
            axes[ind].xaxis.set_ticks(range(len(errors_train[param]['params'])), errors_train[param]['params'])
        axes[ind].set_title(str(param))
    plt.legend()
    plt.tight_layout()

In [22]:
rf_params = {"n_estimators": [10,50,150,300,450,600], 
             "max_depth": range(1,26, 2), 
             "min_samples_split": range(1,26, 2), 
             "min_samples_leaf": range(1,26, 2), 
             "max_features": range(10, 100, 10)}

get_best_params(GradientBoostingClassifier(), rf_params, train_data)

00:00:43
n_estimators 10
Model: GradientBoostingClassifier Metric: log_loss On test: 1.067229
00:04:04
n_estimators 50
Model: GradientBoostingClassifier Metric: log_loss On test: 0.665445


KeyboardInterrupt: 

In [5]:
target = train_data['target'].values
train = train_data.drop(['target'], axis = 1)

In [6]:
test = pd.read_csv('Data/test.csv')

In [7]:
test = test.drop("id", axis = 1)

In [11]:
from sklearn.ensemble import ExtraTreesClassifier
clf = ExtraTreesClassifier(n_estimators=200, min_samples_leaf=10, max_features=30, criterion="gini", min_samples_split=3, n_jobs=4)
clf.fit(train, target)
predicted = clf.predict_proba(test)

In [12]:
sub = pd.DataFrame(predicted, columns = ['Class_1','Class_2','Class_3','Class_4','Class_5','Class_6','Class_7','Class_8','Class_9'])

In [13]:
sub["id"] = list(range(1,len(sub.index) + 1))
sub.head()

Class_1   Class_2   Class_3   Class_4   Class_5   Class_6   Class_7  \
0  0.016149  0.298917  0.237240  0.348643  0.003502  0.023315  0.044615   
1  0.020050  0.087136  0.032367  0.015229  0.005956  0.533094  0.021089   
2  0.002127  0.001288  0.001146  0.000858  0.001970  0.948176  0.008132   
3  0.005149  0.555534  0.275743  0.067138  0.000278  0.005573  0.008840   
4  0.103240  0.007701  0.004552  0.000625  0.000417  0.043724  0.036427   

    Class_8   Class_9  id  
0  0.015468  0.012150   1  
1  0.251825  0.033253   2  
2  0.034155  0.002148   3  
3  0.007268  0.074479   4  
4  0.397412  0.405901   5

In [14]:
cols = sub.columns.tolist()
cols = cols[-1:] + cols[:-1]
sub = sub[cols]

In [15]:
sub.head()

id   Class_1   Class_2   Class_3   Class_4   Class_5   Class_6   Class_7  \
0   1  0.016149  0.298917  0.237240  0.348643  0.003502  0.023315  0.044615   
1   2  0.020050  0.087136  0.032367  0.015229  0.005956  0.533094  0.021089   
2   3  0.002127  0.001288  0.001146  0.000858  0.001970  0.948176  0.008132   
3   4  0.005149  0.555534  0.275743  0.067138  0.000278  0.005573  0.008840   
4   5  0.103240  0.007701  0.004552  0.000625  0.000417  0.043724  0.036427   

    Class_8   Class_9  
0  0.015468  0.012150  
1  0.251825  0.033253  
2  0.034155  0.002148  
3  0.007268  0.074479  
4  0.397412  0.405901

In [16]:
sub.to_csv('Data/submission.csv', index = False)

In [17]:
from sklearn.base import BaseEstimator
from sklearn.base import ClassifierMixin
from sklearn.base import TransformerMixin
from sklearn.preprocessing import LabelEncoder
from sklearn.externals import six
from sklearn.base import clone
import numpy as np
import operator


class EnsembleClassifier(BaseEstimator, ClassifierMixin, TransformerMixin):

    def __init__(self, clfs, voting='hard', weights=None, fitted = False):
        
        self.clfs = clfs
        self.voting = voting
        self.weights = weights
        self.fitted = fitted
        
    def fit(self, X, y):
        
        if isinstance(y, np.ndarray) and len(y.shape) > 1 and y.shape[1] > 1:
            raise NotImplementedError('Multilabel and multi-output'\
                                      ' classification is not supported.')

        if self.voting not in ('soft', 'hard'):
            raise ValueError("Voting must be 'soft' or 'hard'; got (voting=%r)"
                             % voting)

        if self.weights and len(self.weights) != len(self.clfs):
            raise ValueError('Number of classifiers and weights must be equal'
                             '; got %d weights, %d clfs'
                             % (len(self.weights), len(self.clfs)))
        

        self.le_ = LabelEncoder()
        self.le_.fit(y)
        self.classes_ = self.le_.classes_
        self.clfs_ = []
        
        if self.fitted == True:
            for clf in self.clfs:
                self.clfs_.append(clone(clf))
            return self
        
        for clf in self.clfs:
            fitted_clf = clone(clf).fit(X, self.le_.transform(y))
            self.clfs_.append(fitted_clf)
        return self

    def predict(self, X):

        if self.voting == 'soft':

            maj = np.argmax(self.predict_proba(X), axis=1)

        else:  # 'hard' voting
            predictions = self._predict(X)

            maj = np.apply_along_axis(
                                      lambda x:
                                      np.argmax(np.bincount(x,
                                                weights=self.weights)),
                                      axis=1,
                                      arr=predictions)
            
        maj = self.le_.inverse_transform(maj)
        return maj

    def predict_proba(self, X):

        avg = np.average(self._predict_probas(X), axis=0, weights=self.weights)
        return avg

    def transform(self, X):

        if self.voting == 'soft':
            return self._predict_probas(X)
        else:
            return self._predict(X)

    def _predict(self, X):
        if self.fitted == True:
            return np.asarray([clf.predict(X) for clf in self.clfs]).T
        return np.asarray([clf.predict(X) for clf in self.clfs_]).T

    def _predict_probas(self, X):
        if self.fitted == True:
            return np.asarray([clf.predict_proba(X) for clf in self.clfs])
        return np.asarray([clf.predict_proba(X) for clf in self.clfs_])

In [20]:
clf1 = LogisticRegression()
clf2 = RandomForestClassifier(n_estimators=400, 
                              max_depth=15, 
                              min_samples_split=12, 
                              min_samples_leaf=3, 
                              max_features=20, 
                              n_jobs=4)

clf3 = ExtraTreesClassifier(n_estimators=200, 
                            min_samples_leaf=10, 
                            max_features=30, 
                            criterion="gini", 
                            min_samples_split=3, 
                            n_jobs=4)

essemble_classifier = EnsembleClassifier(clfs=[clf1, clf2, clf3], voting='soft')
essemble_classifier.fit(train, target)
predicted = essemble_classifier.predict(test)

In [22]:
predicted = essemble_classifier.predict_proba(test)
sub = pd.DataFrame(predicted, columns = ['Class_1','Class_2','Class_3','Class_4','Class_5','Class_6','Class_7','Class_8','Class_9'])
sub["id"] = list(range(1,len(sub.index) + 1))
cols = sub.columns.tolist()
cols = cols[-1:] + cols[:-1]
sub = sub[cols]
sub.to_csv('Data/submission.csv', index = False)